# PiPP wh-effects

In [1]:
__author__ = "Christopher Potts"

## Set-up

In [2]:
import json
import glob
import matplotlib.pyplot as plt
import numpy as np
import openai
import os
import pandas as pd
import re

import utils

In [3]:
plt.style.use("pipp.mplstyle")

In [4]:
openai_key = os.getenv('OPENAI_API_KEY')

## Items

In [5]:
utils.item("Happy* though we were GAP with the idea, we had to reject it.", 
     embedding="", 
     preposition="though")

{'PiPP (Filler/Gap)': ('Happy though we were with the idea, we had to reject it.',
  'with'),
 'PP (No Filler/No Gap)': ('Though we were happy with the idea, we had to reject it.',
  'happy'),
 'Filler/No Gap': ('Happy though we were happy with the idea, we had to reject it.',
  'happy'),
 'No Filler/Gap': ('Though we were with the idea, we had to reject it.',
  'with')}

## Materials

In [6]:
with open("materials.txt") as f:
    materials = f.read().splitlines()

## Experiments

In [7]:
def run_item(ex, item_num, model_func, embedding="", preposition="though"):
    data = []
    conds = utils.item(ex, embedding=embedding, preposition=preposition)
    for typ, (text, target) in conds.items():
        response = model_func([text])
        ti = [i for i, tok in enumerate(response['prompt_tokens']) if tok.strip() == target][0]
        surprisal = convert_to_surprisal(response['prompt_scores'][ti])
        response['condition'] = typ
        response['target_surprisal'] = surprisal
        response['item_num'] = item_num
        data.append(response)
    return data

def convert_to_surprisal(x):
    return -(x / np.log(2))

In [8]:
def run_experiment(materials, model_func, embedding="", preposition="though", model_name="text-davinci-003"):
    data = []
    for item_num, m in enumerate(materials, start=1):
        data += run_item(m, item_num, model_func, embedding=embedding, preposition=preposition)
    emb = f"-{embedding.replace(' ', '_')}" if embedding else ""
    output_filename = f"results/results-{model_name}-{preposition}{emb}.json"
    with open(output_filename, "wt") as f:
        json.dump(data, f, indent=4)

### GPT-3

In [9]:
for engine in ("ada", "text-davinci-001", "text-davinci-003"):
    for embedding in ("", "they said that we knew that"):
        for prep in ("though", "as", "asas"):
            model_func = (lambda prompts: utils.run_gpt3(prompts, engine=engine))
            run_experiment(materials, model_func, embedding=embedding, preposition=prep, model_name=engine)

### GPT-2

In [10]:
gpt2_tokenizer, gpt2_model = utils.load_hugging_face_model("gpt2-large")

In [11]:
for embedding in ("", "they said that we knew that"):
    for prep in ("though", "as", "asas"):
        model_func = (lambda prompts: utils.run_hugging_face_autoregressive(prompts, gpt2_tokenizer, gpt2_model))
        run_experiment(materials, model_func, embedding=embedding, preposition=prep, model_name="gpt2-large")

## Visualization

In [12]:
def mean_plot(filename, xlim=14):
    df = pd.read_json(filename)

    order = ['PiPP (Filler/Gap)',  'No Filler/Gap', 'Filler/No Gap', 'PP (No Filler/No Gap)'][::-1]

    mus = df.groupby("condition")['target_surprisal'].mean()
    mus = mus.loc[order]

    cis = df.groupby("condition")['target_surprisal'].apply(utils.get_cis)
    cis = cis.loc[order]
    cis = np.array([np.array(x) for x in cis.values])

    gapless_color = "#a1def0"
    gap_color = "#881d2a"

    ax = mus.plot.barh(
        xerr=cis.T,
        figsize=(8, 5),
        color=[gapless_color, gapless_color, gap_color, gap_color])

    ax.set_xlabel("Mean surprisal")
    ax.set_ylabel("")
    ax.set_xlim([0, xlim])

    add_wh_effect(ax, mus.iloc[1], mus.iloc[0], 0.5, gapless_color)
    add_wh_effect(ax, mus.iloc[3], mus.iloc[2], 2.5, gap_color)

    output_filename = filename.replace(".json", ".pdf")
    plt.tight_layout()
    plt.savefig(output_filename, dpi=500)
    plt.close()


def add_wh_effect(ax, v1, v2, yval, color, nudge=0.2):
    effect = v1 - v2
    xpos = max([v1, v2]) + nudge
    val = "${}{:.2f}$".format("+" if effect > 0 else "", effect)
    ax.plot([v2, v1], [yval, yval], lw=3, linestyle="dotted", color=color)
    ax.text(xpos, yval, val, va='center', color='black', fontsize=16, ha='left')

In [13]:
for results_filename in glob.glob(os.path.join("results", "*.json")):
    xlim = 24 if "davinci" in results_filename else 20
    mean_plot(results_filename, xlim=xlim)